<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Map reduce

# Simplistic way of counting words
from typing import List
from collections import Counter

def tokenize(document: str) -> List[str]:
  """Just split on whitespace"""
  return document.split()

def word_count_old(documents: List[str]):
  """Word count not using MapReduce"""
  return Counter(word
                 for document in documents
                 for word in tokenize(document))